In [1]:
import pandas as pd
import basicdescriptives_mod as bd
import re
import string
from unidecode import unidecode
from fuzzywuzzy import fuzz
import numpy as np
import locale

In [2]:
df = pd.read_csv('../data/stage/db_stage_infousers.csv')

## Manejo datos de fecha


In [3]:
df['FECHA DESEMBOLSO'] = df['FECHA DESEMBOLSO'].str.replace('/', '-')
df['FECHA DESEMBOLSO'] = pd.to_datetime(df['FECHA DESEMBOLSO'], errors='coerce', format='%m-%d-%Y')

In [4]:
df['PRÓXIMA FECHA PAGO'] = df['PRÓXIMA FECHA PAGO'].str.replace('/', '-')
df['PRÓXIMA FECHA PAGO'] = pd.to_datetime(df['PRÓXIMA FECHA PAGO'], errors='coerce', format='%m-%d-%Y')

## Procesamiento de texto variables categricas


In [5]:
def processing_text(texto):
    if pd.isna(texto):
        return np.nan
    texto = str(texto)
    ### Limpieza
    texto = texto.lower() # Estandarizar todo a minúscula
    texto = unidecode(texto)  # Eliminar tildes
    texto = re.sub(r'[^a-zA-Z0-9\s/-]', '', texto)
    texto = re.sub(r'\s+', ' ', texto) # Eliminar espacios en blanco adicionales
    texto = texto.strip() # Eliminar espacios al inicio y al final
    
    return texto  

In [6]:
df['CIUDAD RESIDENCIA'].nunique()

726

In [7]:
categorical_columns = df.select_dtypes(include=['object']).columns
df[categorical_columns] = df[categorical_columns].map(processing_text)

## agrupacion de ciudades


In [8]:
departamentos_colombia = [
    "amazonas",
    "antioquia",
    "arauca",
    "atlantico",
    "bolivar",
    "boyaca",
    "caldas",
    "caqueta",
    "casanare",
    "cauca",
    "cesar",
    "choco",
    "cordoba",
    "cundinamarca",
    "guainia",
    "guaviare",
    "huila",
    "la guajira",
    "magdalena",
    "meta",
    "narino",
    "norte de santander",
    "putumayo",
    "quindio",
    "risaralda",
    "san andres y providencia",
    "santander",
    "sucre",
    "tolima",
    "valle", "cauca",
    "vaupes",
    "vichada"
]


In [9]:
## procesamiento de texto extra para las ciudades
def processing_text_ciudades(texto):
    if pd.isna(texto):
        return np.nan
    texto = str(texto)
    ### Limpieza
    texto = texto.lower() # Estandarizar todo a minúscula
    texto = re.sub(r'\s+', ' ', texto) # Eliminar espacios en blanco adicionales
    texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto) # Eliminar caracteres especiales
    texto = texto.strip() # Eliminar espacios al inicio y al final
    
    # Tokenizar el texto
    palabras = texto.split()
    
    # Filtrar las palabras que no son nombres de departamentos
    palabras_filtradas = [palabra for palabra in palabras if not any(fuzz.ratio(palabra, depto) >= 80 for depto in departamentos_colombia)]
    
    # Reconstruir el texto
    texto_filtrado = ' '.join(palabras_filtradas)
    if not texto_filtrado:  # Si el texto filtrado está vacío
        return np.nan
    
    return texto_filtrado


In [10]:
df['CIUDAD RESIDENCIA'] = df['CIUDAD RESIDENCIA'].apply(processing_text_ciudades)

In [11]:
def mapear_ciudades(df):
    mapeo = {}

    for ciudad in df['CIUDAD RESIDENCIA']:
        if isinstance(ciudad, str):  # Verificar si es un string
            ciudad_estandar = None
            # Buscar si la ciudad ya está en el diccionario
            for ciudad_mapeada in mapeo:
                if fuzz.ratio(ciudad.lower(), ciudad_mapeada.lower()) > 70:  # Umbral de similitud del 80%
                    ciudad_estandar = ciudad_mapeada
                    break
            if ciudad_estandar:
                mapeo[ciudad_estandar].append(ciudad)
            else:
                mapeo[ciudad] = [ciudad]

    return mapeo

def obtener_ciudad_estandar(ciudad, mapeo):
    if pd.isna(ciudad):
        return np.nan
    if isinstance(ciudad, str):  # Verificar si es un string
        for ciudad_estandar, variantes in mapeo.items():
            if ciudad in variantes:
                return ciudad_estandar
    return ciudad

# Crear el mapeo
mapeo = mapear_ciudades(df)

# Crear una nueva columna en el DataFrame con la ciudad estandarizada
df['ciudad_estandarizada'] = df['CIUDAD RESIDENCIA'].apply(lambda x: obtener_ciudad_estandar(x, mapeo))


In [12]:
df['ciudad_estandarizada'].nunique()

255

In [13]:
df_ciudades = pd.read_csv('../data/raw/ciudades.csv')

In [14]:
def check_similarity(x, cities_list):
    if not isinstance(x, str) or pd.isnull(x):
        return False
    return any(fuzz.ratio(x, city) >= 70 for city in cities_list)

In [15]:
df['En_lista'] = df['ciudad_estandarizada'].apply(check_similarity, cities_list=df_ciudades['name'].values)

In [16]:
df['En_lista'].dropna().value_counts()

En_lista
True     4035
False      70
Name: count, dtype: int64

In [17]:
df[['ciudad_estandarizada','En_lista']][df['En_lista']==False].info()

<class 'pandas.core.frame.DataFrame'>
Index: 70 entries, 12 to 4084
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ciudad_estandarizada  26 non-null     object
 1   En_lista              70 non-null     bool  
dtypes: bool(1), object(1)
memory usage: 1.2+ KB


In [18]:
df[['ciudad_estandarizada','En_lista']][df['En_lista']==False].dropna().head(30)

,ciudad_estandarizada,En_lista
23,san pedro de los milagros,False
204,cundaytolima,False
220,fundacionmagdalena,False
388,cundaytolima,False
1231,bogota distrito capital,False
1247,3015554054,False
1516,11001,False
1616,san andres de cuerquia,False
1622,calivalle,False
1925,manatiatlantico,False


In [19]:
def obtener_ubicacion_ciudad(ciudad, ciudades_df, umbral_similitud):
    if pd.isna(ciudad):  # Si la ciudad es NaN, devolver None
        return None
    
    for index, row in ciudades_df.iterrows():
        if fuzz.ratio(ciudad, row['name']) >= umbral_similitud:
            return row['location']
    return np.nan

In [20]:
df['Ubicacion'] = df['ciudad_estandarizada'].apply(lambda x: obtener_ubicacion_ciudad(x, df_ciudades, 80))

In [21]:
regex = r'"latitude":(-?\d+\.\d+),"longitude":(-?\d+\.\d+)'
def extraer_latitud(ubicacion):
    if pd.isna(ubicacion):
        return None
    match = re.search(regex, ubicacion)
    if match:
        return float(match.group(1))
    else:
        return None

# Función para extraer longitud
def extraer_longitud(ubicacion):
    if pd.isna(ubicacion):
        return None
    match = re.search(regex, ubicacion)
    if match:
        return float(match.group(2))
    else:
        return None

# Crear columnas de latitud y longitud
df['latitud'] = df['Ubicacion'].apply(extraer_latitud)
df['longitud'] = df['Ubicacion'].apply(extraer_longitud)

In [22]:
df[['ciudad_estandarizada', 'latitud','longitud']].head(20)

,ciudad_estandarizada,latitud,longitud
0,pasto,1.21361,-77.28111
1,buga,3.90089,-76.29783
2,cucuta,7.53941,-72.77238
3,bogota,4.60971,-74.08175
4,la ceja,5.04333,-76.01667
5,cucuta,7.53941,-72.77238
6,campoalegre,2.68489,-75.32311
7,medellin,6.25184,-75.56359
8,pereira,4.81333,-75.69611
9,bogota,4.60971,-74.08175


In [23]:
df['latitud'].isnull().sum()

138

## Procesamiento tiempo en trabajo


In [24]:
def agregar_anios(valor):
    if str(valor).isnumeric():
        return str(valor) + ' anos'
    return valor

# Aplicar la función a la columna 'EDAD'
df['TIEMPO TRABAJO'] = df['TIEMPO TRABAJO'].apply(agregar_anios)

In [25]:
# Separar datos con letras y números
df['TIEMPO TRABAJO'] = df['TIEMPO TRABAJO'].fillna('nan')
letras_numeros = df['TIEMPO TRABAJO'].str.contains(r'[a-zA-Z]', regex=True)
#letras_numeros = letras_numeros.fillna('nan')  # Rellenar NaN con False
df['letras_numeros'] = df['TIEMPO TRABAJO'][letras_numeros]
df['letras_numeros'] = df['letras_numeros'].replace('nan', np.nan)
df['fechas_trab'] = df['TIEMPO TRABAJO'][~letras_numeros]

In [26]:
def es_fecha(texto):
    if pd.isna(texto):
        return np.nan
    texto = re.sub(r'(\D)(\d)', r'\1 \2', texto)
    texto = re.sub(r'[^a-zA-Z0-9\s]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    
    
    meses = r'(enero|febrero|marzo|abril|mayo|junio|julio|agosto|septiembre|octubre|noviembre|diciembre)'
    patron = rf'({meses} \d{{1,2}} de \d{{4}}|{meses} \d{{1,2}} \d{{4}}|{meses} – \d{{1,2}} – \d{{4}}|{meses}\d{{1,2}}-\d{{4}})'
    match = re.search(patron, texto)
    if match:
        return match.group()

df['fechas'] = df['letras_numeros'].apply(es_fecha)

In [27]:
df['fechas'].dropna().head()

262       marzo 1 de 2019
315          mayo 15 2018
2895       agosto 20 2000
3198    noviembre 19 2018
3719         julio 2 2015
Name: fechas, dtype: object

In [28]:
def obtener_valor_numerico(texto):
    if texto is not np.nan:
        texto = str(texto)
        texto = texto.lower()
        meses = 0
        anios = 0
        if 'mes' in texto:
            meses = re.findall(r'(\d+)\s*(?:mes?|meses?)', texto)
            if len(meses)==0:
                meses = 1
            else:    
                meses = int(meses[0])
                
        if any(word in texto.lower() for word in ['años', 'anos', 'año', 'años', 'ano']):
            anios = re.findall(r'(\d+)\s*(?:años?|anos?|año)', texto)
            if len(anios)==0:
                anios = 12
            else:    
                anios = 12*int(anios[0])
        if 'medio' in texto:
            meses = meses + 6
        return meses + anios
    return np.nan

In [29]:
df['letras_numeros'] = df['letras_numeros'].apply(obtener_valor_numerico)

In [30]:
df[['TIEMPO TRABAJO','letras_numeros']].head(5)

,TIEMPO TRABAJO,letras_numeros
0,24 meses,24.0
1,12 meses,12.0
2,1 ano,12.0
3,nan,NaN
4,9 horas,0.0


In [31]:
df[['TIEMPO TRABAJO','fechas_trab']].dropna().head(5)

,TIEMPO TRABAJO,fechas_trab
19,2018-01-15,2018-01-15
51,2021-03-01,2021-03-01
53,2020-02-04,2020-02-04
76,2019-05-27,2019-05-27
77,2019-07-23,2019-07-23


In [32]:
df['fechas_trab'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4105 entries, 0 to 4104
Series name: fechas_trab
Non-Null Count  Dtype 
--------------  ----- 
2529 non-null   object
dtypes: object(1)
memory usage: 32.2+ KB


In [33]:
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

df['fechas'] = df['fechas'].str.replace(' de ', ' ').str.capitalize()
df['fechas'] = pd.to_datetime(df['fechas'], format='%B %d %Y') 

In [34]:
df['fechas_trab'] = df['fechas_trab'].str.replace('/', '-')
df['TIEMPO TRABAJO FORMATO 1'] = pd.to_datetime(df['fechas_trab'], format='%Y-%m-%d', errors='coerce')
df['TIEMPO TRABAJO FORMATO 2'] = pd.to_datetime(df['fechas_trab'], format='%d-%m-%Y', errors='coerce')

In [35]:
df['TIEMPO TRABAJO FORMATO 2'] = df['TIEMPO TRABAJO FORMATO 2'].dt.strftime('%Y-%m-%d')

In [36]:
df['fechas_trab'] = df['TIEMPO TRABAJO FORMATO 1'].fillna(df['TIEMPO TRABAJO FORMATO 2'])

In [37]:
df['fechas_trab'] = df['fechas_trab'].fillna(df['fechas'])

In [38]:
df['meses_transcurridos'] = ((df['FECHA DESEMBOLSO'] - df['fechas_trab']) / pd.Timedelta(days=30.44))
df['meses_transcurridos'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4105 entries, 0 to 4104
Series name: meses_transcurridos
Non-Null Count  Dtype  
--------------  -----  
2482 non-null   float64
dtypes: float64(1)
memory usage: 32.2 KB


In [39]:
def convertir_a_nan(valor):
    if pd.isnull(valor) or valor < 0:
        return np.nan
    return valor

In [40]:
df['meses_transcurridos'] = df['meses_transcurridos'].apply(convertir_a_nan)
df['meses_transcurridos'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 4105 entries, 0 to 4104
Series name: meses_transcurridos
Non-Null Count  Dtype  
--------------  -----  
2234 non-null   float64
dtypes: float64(1)
memory usage: 32.2 KB


In [41]:
df['meses_transcurridos'] = df['meses_transcurridos'].fillna(df['letras_numeros'])

In [42]:
df[['FECHA DESEMBOLSO','TIEMPO TRABAJO','meses_transcurridos']].head(20)



,FECHA DESEMBOLSO,TIEMPO TRABAJO,meses_transcurridos
0,2015-06-18,24 meses,24.0
1,2015-06-18,12 meses,12.0
2,2015-06-10,1 ano,12.0
3,2015-06-16,nan,NaN
4,2015-06-17,9 horas,0.0
5,2015-06-16,3 anos,36.0
6,2015-06-16,nan,NaN
7,2015-06-26,3 meses,3.0
8,2015-06-23,1 ano,12.0
9,2015-06-23,nan,NaN


In [43]:
df.columns

Index(['PLAZO', 'CAPITAL', 'INT CORRIENTE', 'DÍAS MORA', 'Cuotas en mora',
       'ESTRATO', 'INGRESOS MENSUALES', 'GASTOS MENSUALES',
       'INGRESOS ADICIONALES', 'PERSONAS A CARGO', 'NUMERO DE HIJOS',
       'NUM.CREDITOS SOLICITADOS', 'FECHA DESEMBOLSO', 'PRÓXIMA FECHA PAGO',
       'TIPO EMPLEO', 'CIUDAD RESIDENCIA', 'TRABAJO', 'TIPO DE VIVIENDA',
       'AÑOS EN LA VIVIENDA', 'TIPO DE CONTRATO', 'PERIODO DE PAGO',
       'ESTADO CIVIL', 'NIVEL EDUCATIVO', 'TIPO DE VEHICULO', 'TIEMPO TRABAJO',
       'ciudad_estandarizada', 'En_lista', 'Ubicacion', 'latitud', 'longitud',
       'letras_numeros', 'fechas_trab', 'fechas', 'TIEMPO TRABAJO FORMATO 1',
       'TIEMPO TRABAJO FORMATO 2', 'meses_transcurridos'],
      dtype='object')

In [44]:
## Columnas a eliminar
columas_to_drop = ['TIEMPO TRABAJO','fechas','fechas_trab','TIEMPO TRABAJO FORMATO 1',
                   'TIEMPO TRABAJO FORMATO 2','letras_numeros', 'FECHA DESEMBOLSO',
                   'CIUDAD RESIDENCIA','Ubicacion','TRABAJO','ciudad_estandarizada', 'En_lista',]
df.drop(columas_to_drop, axis=1, inplace=True)

## one hot encoding


In [45]:
columnas_to_encode = ['TIPO EMPLEO', 'TIPO DE VIVIENDA',
       'TIPO DE CONTRATO', 'PERIODO DE PAGO', 'ESTADO CIVIL',
       'NIVEL EDUCATIVO', 'TIPO DE VEHICULO']

In [46]:
df_dummies = pd.get_dummies(df, columns=columnas_to_encode, drop_first=True,dummy_na=True,dtype=float)
nan_df = df_dummies.loc[:, df_dummies.columns.str.endswith("_nan")]

pattern = "^([^_]*)_"
regex = re.compile(pattern)

for index in df_dummies.index:
    for col_nan in nan_df.columns:
        if df_dummies.loc[index,col_nan] == 1:
            col_id = regex.search(col_nan).group(1)
            targets = df_dummies.columns[df_dummies.columns.str.startswith(col_id+'_')]
            df_dummies.loc[index, targets] = np.nan


In [47]:
df_final = df_dummies.drop(df_dummies.columns[df_dummies.columns.str.endswith("_nan")], axis=1)

# Encode de columna años en la vivienda


In [48]:
df['AÑOS EN LA VIVIENDA'].unique()

array(['mas de 5 anos', 'entre uno y cinco anos', 'menos de un ano', nan],
      dtype=object)

In [49]:
mapeo = {'mas de 5 anos': 3, 'entre uno y cinco anos': 2, 'menos de un ano': 1}

# Aplicar el mapeo
df_final['AÑOS EN LA VIVIENDA'] = df['AÑOS EN LA VIVIENDA'].map(mapeo)

## Codificacion fecha desembolso a meses

In [50]:
df_final['mes_de_pago'] = df_final['PRÓXIMA FECHA PAGO'].dt.month

In [51]:
df_final[['PRÓXIMA FECHA PAGO','mes_de_pago']].head(20)

,PRÓXIMA FECHA PAGO,mes_de_pago
0,2015-06-27,6
1,2015-07-02,7
2,2015-07-10,7
3,2015-07-15,7
4,2015-07-15,7
5,2015-07-16,7
6,2015-07-16,7
7,2015-07-17,7
8,2015-07-22,7
9,2015-07-23,7


In [52]:
df_final = df_final.drop(columns=['PRÓXIMA FECHA PAGO'], axis=1)

# Guardado del df resultante

In [53]:
df_final.columns

Index(['PLAZO', 'CAPITAL', 'INT CORRIENTE', 'DÍAS MORA', 'Cuotas en mora',
       'ESTRATO', 'INGRESOS MENSUALES', 'GASTOS MENSUALES',
       'INGRESOS ADICIONALES', 'PERSONAS A CARGO', 'NUMERO DE HIJOS',
       'NUM.CREDITOS SOLICITADOS', 'AÑOS EN LA VIVIENDA', 'latitud',
       'longitud', 'meses_transcurridos', 'TIPO EMPLEO_empleado',
       'TIPO EMPLEO_independiente', 'TIPO EMPLEO_pensionado',
       'TIPO EMPLEO_prestador de servicios', 'TIPO DE VIVIENDA_familiar',
       'TIPO DE VIVIENDA_propia', 'TIPO DE CONTRATO_prestacion de servicios',
       'TIPO DE CONTRATO_termino fijo', 'PERIODO DE PAGO_quincenal',
       'ESTADO CIVIL_comprometido/a', 'ESTADO CIVIL_divorciado/a',
       'ESTADO CIVIL_soltero/a', 'ESTADO CIVIL_union libre',
       'ESTADO CIVIL_viudo/a', 'NIVEL EDUCATIVO_postgrado',
       'NIVEL EDUCATIVO_primaria', 'NIVEL EDUCATIVO_profesional',
       'NIVEL EDUCATIVO_secundaria', 'NIVEL EDUCATIVO_tecnico',
       'NIVEL EDUCATIVO_tecnologo', 'TIPO DE VEHICULO_moto'

In [55]:
ruta1 = '../data/stage/db_pross_info_users.csv'
df_final.to_csv(ruta1, index=False) 